## Chapter 8: Number Theory

This chapter uses number theory to investigate more ideas about scientific computation.  We'll solve some problems related to prime numbers and other interesting this in number theory like perfect numbers and happy numbers. This chapter is used as an example of some of the ideas in scientific computing applied to number theory.

### 8.1: Prime Numbers

Recall that a prime number is an integer greater than 1 whose only factors are 1 and itself.

In [ ]:
function findAllFactors(n::Integer) 
  local factors = [1]
  for i=2:n-1
    if mod(n,i)==0 
      push!(factors,i)
    end
  end
  push!(factors,n) # n is always a factor of itself 
end

This function finds all of the factors of an integer. The result is an array of all of the factors: 

In [ ]:
findAllFactors(24)

In [ ]:
findAllFactors(50)

Try to find a function `isPrime` that determines if its input is prime or not:

In [ ]:
function isPrime(n::Integer)
  factors = findAllFactors(n)
  if length(factors) == 2
    return true
  else
    return false
  end
end

In [ ]:
isPrime(13)

In [ ]:
isPrime(15)

Write a more compact version of `isPrime`

In [ ]:
function isPrime(n::Integer)

end

### 8.2: Finding Primes

It's also helpful to perhaps have a function that produce the next prime given any number:

In [ ]:
nextPrime(n::Integer) = isPrime(n+1) ? n+1 : nextPrime(n+1)

This is recursive.  It first's test if the next integer `n+1` is prime, if so, return `n+1`, else return `nextPrime(n+1)`

Let's test this a bit:

In [ ]:
nextPrime(10)

In [ ]:
nextPrime(47)

In [ ]:
nextPrime(1_000_000)

In [ ]:
function nextPrime(n::Integer)
    while !isPrime(n+1)
        n+=1
    end
    n+1    
end

In [ ]:
nextPrime(47)

In [ ]:
nextPrime(1_000_000)

In [ ]:
using Primes

In [ ]:
nextprime(1_000_000_000_000)

### 8.3 Perfect Numbers

A number is perfect if the sum of its factors (except itself) is itself.  For example, 6 is perfect because it factors are 1,2,3,6 and the sum of the first 3 numbers is 6. We seek to write a function `isPerfect` to test this.

In [ ]:
function isPerfect(n::Integer)
  # find the factors of n and check the sum
end

In [ ]:
isPerfect(6)

In [ ]:
isPerfect(28)

In [ ]:
isPerfect(10)

In [ ]:
filter(isPerfect,1:100)

In [ ]:
isPerfect2(n::Integer) = 

Let's time these two functions to see which is faster.  

In [ ]:
@time isPerfect(10_000)

In [ ]:
@time isPerfect2(10_000)

Notice that `@time` macro is reporting, we're going to use a package called `BenchmarkTools`.  We just did a tutorial for package managing in julia.  If you need a refresher see Appendix B of the textbook.

In [ ]:
using BenchmarkTools

In [ ]:
@btime isPerfect(10_000)

In [ ]:
@btime isPerfect2(10_000)

The `@btime` macro runs the command a number of times and reports the average.  This helps in that since other things are happening on your computer, this is more accurate. Also, is using milliseconds (ms), microseconds (μs) and nanoseconds (ns). 

### Happy Numbers

Here’s another fun example. A number n is called happy if you perform the following steps:
1. Take the digits of n and square each one.
2. Sum the squares.
3. If the sum is 1, then the number is happy. If not repeat these steps.

* 13 is happy
* 19 is happy
* 4 is not happy

In [ ]:
function isHappy(n::Integer)
  if n==1
    return true
  elseif n==4 
    return false
  else 
    local d = digits(n)
    local sum = 0
    for i=1:length(d)
      sum += d[i]^2
    end
    return isHappy(sum)
  end
end

In [ ]:
isHappy(10)

In [ ]:
isHappy(11)

In [ ]:
isHappy(58)

In [ ]:
function isHappy2(n::Integer)
  if n==1
    return true
  elseif n==4 
    return false
  else 
    return isHappy2(sum(x->x^2,digits(n)))
  end
end

In [ ]:
isHappy2(10)

In [ ]:
isHappy2(11)

In [ ]:
function isHappy3(n::Integer)
  n == 1 ? true : (n == 4 ? false : isHappy3(sum(x->x^2,digits(n))))
end

In [ ]:
isHappy3(10)

In [ ]:
isHappy3(11)

You will time this in your homework using the Benchmark tools

Let's find which of the first 100 whole numbers are happy.

In [ ]:
[isHappy(n) for n=1:100]

Although, you can figured it out a bit (the 1s mean the number is happy, 0 means it's not).  The following is a big easier:

In [ ]:
filter(isHappy, 1:100)

Remember what just happened here.  The numbers 1 to 100 (as a range) are filtered with only those that are true with the `isHappy` command.

### 8.5: Primes can be big!!

The following is a prime number:

In [ ]:
n = big(2)^89-1

which doesn't fit in `Int64`, which is why we used a `BigInt`.   Don't try to run `isPrime` on this number though. Trust me it won't finish because our algorithm isn't very good.  The largest known prime (as of mid 2020) is 24,862,048 digits long. 

### 8.6 speeding up our algorithms

If we benchmark time our original `isPrime` function on a reasonably sized prime number (but 1 million is tiny in prime number land), we get:

In [ ]:
using BenchmarkTools

In [ ]:
@btime isPrime(1_000_003)

#### Speeding up `findAllFactors`

Since `isPrime` just uses `findAllFactors`, we should first concentrate on improving `findAllFactors`.  First, we note that we don't need to check all numbers for factors.  For a given number $n$, the biggest factor (other than $n$), can be $n/2$.  Let's stop at this point:

In [ ]:
function findAllFactors2(n::Integer)
  factors = [1]
  for i=2:n/2
    if mod(n,i)==0
      push!(factors,i)
    end
  end
  push!(factors,n)
end

Let's time this versus the previous `findAllFactors`:

In [ ]:
@btime findAllFactors(10_000);

Note that we have supressed the output using the ; because we're more interested in the timing of the function rather than the result.

In [ ]:
@btime findAllFactors2(10_000);

This may be surprising that it is slower.  The reason why is because if we do `n/2`, the result is a Float and all calculations with floating points are always slower than with integers. Let's replace `n/2` with integer division:

In [ ]:
function findAllFactors3(n::Integer)
  factors = [1]
  for i=2:div(n,2)
    if mod(n,i)==0
      push!(factors,i)
    end
  end
  push!(factors,n)
end

In [ ]:
@btime findAllFactors3(10_000);

Note that now we has sped up the result by about a factor of 50%, which is to be expected because we've cut down on number of checks. 

#### keep eliminating factors:

Factors nearly always come in pairs.  Consider all factors of 200:

```
1    2    4   5   8   10  
200  100  50  40  25  20
```


In [ ]:
function findAllFactors4(n::Integer)
  local x = round(Int,sqrt(n)) # closest integer to sqrt(n)
  local factors = [1,n]
  local j=2 # keep track where to insert elements
  for k=2:x
    if n%k==0
      # insert the new factors in the middle of the factors array
      splice!(factors,j:(j-1),[k,div(n,k)])
      j+=1
    end
  end
  unique(factors)
end

In [ ]:
findAllFactors4(200)

In [ ]:
@btime findAllFactors4(10_000);

This shows a speed up of about 12 for this example.  This doesn't scale linearly because we stop at $\sqrt{n}$

Let's now check the various `isPrime` functions

In [ ]:
function isPrime(n::Integer)
  return length(findAllFactors(n))==2
end

In [ ]:
function isPrime2(n::Integer)
  return length(findAllFactors3(n))==2
end

In [ ]:
function isPrime3(n::Integer)
  return length(findAllFactors4(n))==2
end

In [ ]:
@btime isPrime(1_000_003)

In [ ]:
@btime isPrime2(1_000_003)

In [ ]:
@btime isPrime3(1_000_003)

#### if we are determining primeness, why find all factors

Let's just walk through all integers up to $\sqrt{n}$ and stop if we find a factor

In [ ]:
function isPrime4(n::Integer)
  if n==1
    return false
  end
  local x = round(Int,sqrt(n)) # find integer nearest sqrt(n)
  for k=2:x
    if n%k==0
      return false
    end
  end
  true
end

Let's check this with a larger prime:

In [ ]:
@btime isPrime3(100_000_007)

In [ ]:
@btime isPrime4(100_000_007)

These look to be nearly the same.  Note that `isPrime3` uses an array but still is about as fast.

We can cut down a bit more, by only checking odd numbers. 

In [ ]:
function isPrime5(n::Integer)
  if n%2==0
    return false
  end
  for k=3:2:round(Int,sqrt(n))
    if n%k==0
      return false
    end
  end
  true
end

In [ ]:
@btime isPrime5(100_000_007)

In [ ]:
function getPrimes(n::Integer) ## return all primes up to n using the sieve of erathones
  local is_prime=trues(n) ## assume all are prime
  local k=2
  while true
    is_prime[2*k:k:n] .= false
    k = findnext(is_prime,k+1) # find the next prime after k
    if isnothing(k)
      return findall(is_prime)[2:end]
    end
  end
end

#### using the Primes package

In [ ]:
using Primes

In [ ]:
@btime isprime(100_000_007)

In [ ]:
@btime isPrime(100_000_007)

In [ ]:
@btime isprime(big(2)^89-1)

In [ ]:
filter(isPrime5,1_000_000_000_000:1_000_000_000_100)